<a href="https://colab.research.google.com/github/Balaji-R1/carprice/blob/main/car_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.feature_selection import RFE, f_regression
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
np.random.seed(0)
np.set_printoptions(precision=2)

In [ ]:
def pretty_print_linear(coefs, names = None, sort = False):
    if names is None:
        names = ["X%s" % x for x in range(len(coefs))]
    lst = zip(coefs, names)
    if sort:
        lst = sorted(lst, key = lambda x:-np.abs(x[0]))
    return " + ".join("%s * %s" % (round(coef, 3), name) for coef, name in lst)

In [ ]:
def root_mean_square_error(y_pred,y_test):
    rmse_train = np.sqrt(np.dot(abs(y_pred-y_test),abs(y_pred-y_test))/len(y_test))
    return rmse_train

In [ ]:
def ohc_conversion(input_text):
    if "ohc" in input_text:
        return 1
    else:
        return 0

In [ ]:
def plot_real_vs_predicted(y_pred,y_test):
    plt.plot(y_pred,y_test,'ro')
    plt.plot([0,50],[0,50], 'g-')
    plt.xlabel('predicted')
    plt.ylabel('real')
    plt.show()
    return plt

Reading the csv file

In [ ]:
df = pd.read_csv('CarPrice_Assignment.csv')

In [ ]:
df.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [ ]:
print ("Number of rows in dataset = "), df.shape[0]
print ("Number of features in dataset = "), df.shape[1]

Number of rows in dataset = 
Number of features in dataset = 


(None, 26)

In [ ]:
df.drop('car_ID',axis=1, inplace=True)

In [ ]:
df.isnull().sum()

symboling           0
CarName             0
fueltype            0
aspiration          0
doornumber          0
carbody             0
drivewheel          0
enginelocation      0
wheelbase           0
carlength           0
carwidth            0
carheight           0
curbweight          0
enginetype          0
cylindernumber      0
enginesize          0
fuelsystem          0
boreratio           0
stroke              0
compressionratio    0
horsepower          0
peakrpm             0
citympg             0
highwaympg          0
price               0
dtype: int64

In [ ]:
df.describe()

,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
count,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000
mean,0.834146,98.756585,174.049268,65.907805,53.724878,2555.565854,126.907317,3.329756,3.255415,10.142537,104.117073,5125.121951,25.219512,30.751220,13276.710571
std,1.245307,6.021776,12.337289,2.145204,2.443522,520.680204,41.642693,0.270844,0.313597,3.972040,39.544167,476.985643,6.542142,6.886443,7988.852332
min,-2.000000,86.600000,141.100000,60.300000,47.800000,1488.000000,61.000000,2.540000,2.070000,7.000000,48.000000,4150.000000,13.000000,16.000000,5118.000000
25%,0.000000,94.500000,166.300000,64.100000,52.000000,2145.000000,97.000000,3.150000,3.110000,8.600000,70.000000,4800.000000,19.000000,25.000000,7788.000000
50%,1.000000,97.000000,173.200000,65.500000,54.100000,2414.000000,120.000000,3.310000,3.290000,9.000000,95.000000,5200.000000,24.000000,30.000000,10295.000000
75%,2.000000,102.400000,183.100000,66.900000,55.500000,2935.000000,141.000000,3.580000,3.410000,9.400000,116.000000,5500.000000,30.000000,34.000000,16503.000000
max,3.000000,120.900000,208.100000,72.300000,59.800000,4066.000000,326.000000,3.940000,4.170000,23.000000,288.000000,6600.000000,49.000000,54.000000,45400.000000


In [ ]:
categorical_columns = list(df.select_dtypes(['object']).columns)

In [ ]:
for cols in categorical_columns:
    print ([cols], " : ", df[cols].unique())

['CarName']  :  ['alfa-romero giulia' 'alfa-romero stelvio' 'alfa-romero Quadrifoglio'
 'audi 100 ls' 'audi 100ls' 'audi fox' 'audi 5000' 'audi 4000'
 'audi 5000s (diesel)' 'bmw 320i' 'bmw x1' 'bmw x3' 'bmw z4' 'bmw x4'
 'bmw x5' 'chevrolet impala' 'chevrolet monte carlo' 'chevrolet vega 2300'
 'dodge rampage' 'dodge challenger se' 'dodge d200' 'dodge monaco (sw)'
 'dodge colt hardtop' 'dodge colt (sw)' 'dodge coronet custom'
 'dodge dart custom' 'dodge coronet custom (sw)' 'honda civic'
 'honda civic cvcc' 'honda accord cvcc' 'honda accord lx'
 'honda civic 1500 gl' 'honda accord' 'honda civic 1300' 'honda prelude'
 'honda civic (auto)' 'isuzu MU-X' 'isuzu D-Max ' 'isuzu D-Max V-Cross'
 'jaguar xj' 'jaguar xf' 'jaguar xk' 'maxda rx3' 'maxda glc deluxe'
 'mazda rx2 coupe' 'mazda rx-4' 'mazda glc deluxe' 'mazda 626' 'mazda glc'
 'mazda rx-7 gs' 'mazda glc 4' 'mazda glc custom l' 'mazda glc custom'
 'buick electra 225 custom' 'buick century luxus (sw)' 'buick century'
 'buick skyhawk' 'b

In [ ]:
len(df.CarName.unique())

147

In [ ]:
def car_type(input_text):
    other_cars = set(['porsche', 'isuzu', 'jaguar', 'alfa-romero', 'chevrolet',
              'vw', 'renault','maxda','mercury', 'Nissan', 'toyouta',
              'vokswagen', 'porcshce'])
    if (len(input_text.split(' '))>1) and (input_text.split(' ')[0] not in other_cars):
        return input_text.split(' ')[0]
    else:
        return 'others'

In [ ]:
df.CarName.apply(car_type).value_counts()

toyota        31
others        30
nissan        17
mazda         15
honda         13
mitsubishi    13
peugeot       11
volvo         11
subaru        10
dodge          9
volkswagen     9
bmw            8
buick          8
audi           7
plymouth       7
saab           6
Name: CarName, dtype: int64

In [ ]:
df.CarName = df.CarName.apply(car_type)

In [ ]:
word2num_mapping = {'one': 1,'two':2,'three':3,'four':4, 'five':5, 'six':6, 'seven':7,
                    'eight':8, 'nine':9, 'twelve':12}

Converting categorical variables to numeric

In [ ]:
df.doornumber.replace(word2num_mapping,inplace=True)
df.doornumber = df.doornumber.astype(int)

In [ ]:
df.cylindernumber.replace(word2num_mapping,inplace=True)
df.cylindernumber = df.cylindernumber.astype(int)

Label encoding for some columns containing categorical variables

In [ ]:
df.fueltype       = df.fueltype.astype('category').cat.codes
df.aspiration     = df.aspiration.astype('category').cat.codes
df.enginelocation = df.enginelocation.astype('category').cat.codes

In [ ]:
df["enginetype"].value_counts()

ohc      148
ohcf      15
ohcv      13
dohc      12
l         12
rotor      4
dohcv      1
Name: enginetype, dtype: int64

In [ ]:
df.enginetype = df.enginetype.apply(ohc_conversion)

In [ ]:
Y = df.price.values
df.drop('price',axis=1,inplace=True)
df = pd.get_dummies(df, columns=["CarName","carbody", "drivewheel", "fuelsystem"], prefix=["make", "body", 
                                                                                           "drive", "fuel"])
df.head()

,symboling,fueltype,aspiration,doornumber,enginelocation,wheelbase,carlength,carwidth,carheight,curbweight,...,drive_fwd,drive_rwd,fuel_1bbl,fuel_2bbl,fuel_4bbl,fuel_idi,fuel_mfi,fuel_mpfi,fuel_spdi,fuel_spfi
0,3,1,0,2,0,88.6,168.8,64.1,48.8,2548,...,0,1,0,0,0,0,0,1,0,0
1,3,1,0,2,0,88.6,168.8,64.1,48.8,2548,...,0,1,0,0,0,0,0,1,0,0
2,1,1,0,2,0,94.5,171.2,65.5,52.4,2823,...,0,1,0,0,0,0,0,1,0,0
3,2,1,0,4,0,99.8,176.6,66.2,54.3,2337,...,1,0,0,0,0,0,0,1,0,0
4,2,1,0,4,0,99.4,176.6,66.4,54.3,2824,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
X = df.values
names = df.columns

Feature Selection using Multiple methods

In [ ]:
ranks = {}
r = {}

def rank_to_dict(ranks, names, order=1):
    minmax = MinMaxScaler()
    ranks = minmax.fit_transform(order*np.array([ranks]).T).T[0]
    ranks = map(lambda x: round(x, 2), ranks)
    return dict(zip(names, ranks ))
 
lr = LinearRegression(normalize=True)
lr.fit(X, Y)
ranks["Linear reg"] = rank_to_dict(np.abs(lr.coef_), names)

ridge = Ridge(alpha=7)
ridge.fit(X, Y)
ranks["Ridge"] = rank_to_dict(np.abs(ridge.coef_), names)

lasso = Lasso(alpha=.05)
lasso.fit(X, Y)
ranks["Lasso"] = rank_to_dict(np.abs(lasso.coef_), names)
 
#stop the search when 5 features are left (they will get equal scores)
rfe = RFE(lr, n_features_to_select=5)
rfe.fit(X,Y)
ranks["RFE"] = rank_to_dict(map(float, rfe.ranking_), names, order=-1)
 
rf = RandomForestRegressor()
rf.fit(X,Y)
ranks["RF"] = rank_to_dict(rf.feature_importances_, names)

f, pval  = f_regression(X, Y, center=True)
ranks["Corr."] = rank_to_dict(f, names)

In [ ]:
for name in names:
    r[name] = round(np.mean([ranks[method][name]for method in ranks.keys()]), 2)
    
methods = sorted(ranks.keys())
ranks["Mean"] = r
methods.append("Mean")

print "\t%s" % "\t".join(methods)
for name in names:
    print ("%s\t%s" % (name, "\t".join(map(str, [ranks[method][name] for method in methods]))))

In [ ]:
mean_plot = pd.DataFrame(list(r.items()),columns=['Feature', 'Mean Ranking'])
mean_plot = mean_plot.sort_values('Mean Ranking', ascending=False)


In [ ]:
mean_plot

,Feature,Mean Ranking


In [ ]:
sns.factorplot(x="Mean Ranking", y="Feature", data = mean_plot, kind="bar", 
               size=14, aspect=1.9, palette='coolwarm')

In [ ]:
# Create linear regression object
linreg = LinearRegression()

# Train the model using the training sets
linreg.fit(X,Y)

print "Linear model: ", pretty_print_linear(linreg.coef_, names, sort = True)

# Predict the values using the model
Y_lin_predict = linreg.predict(X)

# Print the root mean square error 
print "Root Mean Square Error: ", root_mean_square_error(Y_lin_predict,Y)

In [ ]:
plot_real_vs_predicted(Y,Y_lin_predict)